In [89]:
import json
import time
import csv
import os

def process_vrp_file(file_path, result_folder):
    # Load the JSON input from a file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # Extract data from JSON
    distance_matrix = data['distance_matrix']
    time_matrix = data['time_matrices'][0]
    demands = data['demands']
    serve_time = data['serve_time']
    vehicle_capacities = data['vehicle_capacities']
    vehicle_max_weight = data['vehicle_capacities']
    fixed_cost = data['fixed_cost']
    cost_per_km = data['cost_per_km']
    contract_distance = data['contract_distance']
    vehicle_volume = data['vehicle_volume']
    cost_per_minute = data['cost_per_minute']
    variable_cost_per_minute = data['variable_cost_per_minute']
    contract_time = data['contract_time']
    vehicle_average_speed = data['vehicle_average_speed']
    order_volume = data['order_volume']
    vehicle_volumes = data['vehicle_volumes']
    vehicle_contract_code = data['vehicle_contract_code']
    vehicle_max_travel_distance = data['vehicle_max_travel_distance']
    vehicle_max_travel_time = data['vehicle_max_available_time']
    order_types = data['order_type']
    depot = data['depot']
    order_id=data['order_id']
    print(order_id)
    num_vehicles = data['num_vehicles']
    paired_orders = data['paired_order']

    start_time = time.time()

    def select_seed_customers_by_proximity(distance_matrix, num_vehicles):
        depot_index = 0
        distances = [(i, distance_matrix[depot_index][i]) for i in range(1, len(distance_matrix))]
        distances.sort(key=lambda x: x[1])
        return [distances[i][0] for i in range(min(num_vehicles, len(distances)))]

    def calculate_route_cost(route, vehicle_index, distance_matrix, time_matrix):
        total_distance = 0
        total_time = 0

        if len(route) > 0:  
            depot_index = 0
            total_distance += distance_matrix[depot_index][route[0]]  
            total_time += time_matrix[depot_index][route[0]]

        for i in range(len(route) - 1):
            total_distance += distance_matrix[route[i]][route[i + 1]]
            total_time += time_matrix[route[i]][route[i + 1]]
            total_time += serve_time[route[i]]

        if len(route) > 0:  
            total_distance += distance_matrix[route[-1]][depot_index]  
            total_time += time_matrix[route[-1]][depot_index]
            total_time += serve_time[route[-1]]

        excess_distance = max(0, total_distance - contract_distance[vehicle_index])
        variable_distance_cost = total_distance * cost_per_km[vehicle_index]

        if excess_distance > 0:
            variable_distance_cost += excess_distance * cost_per_km[vehicle_index]

        total_cost = (fixed_cost[vehicle_index] +
                      variable_distance_cost +
                      (total_time * cost_per_minute[vehicle_index]))

        return total_cost, total_distance, total_time






    def is_valid_route(route, demands, vehicle_capacities, order_volume, vehicle_index, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
        max_volume = vehicle_volume[vehicle_index]
        available_weight = vehicle_capacities[vehicle_index]
        current_time = 0
        pickup_weight = 0
        dropoff_weight = 0
        current_volume = 0
        l=[]
        delivered = set()
        pickups_done = set()


        for i in range(len(route)):
            if route[i] == 0:
                continue

            current_order_type = order_types[route[i] - 1]


            if current_order_type == "pickup":

                if demands[route[i]] > available_weight:
                    return False

                pickup_weight += demands[route[i]]
                current_volume += order_volume[route[i]]


                available_weight -= demands[route[i]]


                if available_weight <= 0 or current_volume > max_volume or pickup_weight > vehicle_capacities[vehicle_index]:
                    return False

                pickups_done.add(route[i])
                l.append(route[i])
                print(f"pickup_weight{pickup_weight}")



            elif current_order_type == "dropoff":

                dropoff_weight += demands[route[i]]
                current_volume += order_volume[route[i]]

                if demands[route[i]] > available_weight:
                    # print("failing here")
                    return False

                if dropoff_weight > vehicle_capacities[vehicle_index] or available_weight==0:
                    
                    return False

                # if available_weight==0:
                #     return False


                # available_weight-=demands[route[i]]
                
                

                #print(paired_orders)

                paired_pickup = [int(pair[0].split('_')[1]) for pair in paired_orders if pair[1] == f"order_{route[i]}_dropoff"]


                # Ensure that the paired pickup has been completed and visited
                if paired_pickup and paired_pickup[0]  not in l:
                    available_weight+=demands[route[i]]
                    pickup_weight-=demands[paired_pickup[0]]
                    # print(f"Paired pickup order_{paired_pickup[0]}_pickup for dropoff order_{route[i]}_dropoff has not been visited.")
                    
                    return False

                
                delivered.add(route[i])     #here paired ordered drop off done so  update increase availble weight
                l.append(route[i])

                available_weight -=demands[route[i]]
                #pickup_weight -=demands[route[i]]
                # print(f"Paired dropoff completed. available weight updated to {available_weight}")
                                                    # Update availbale weight
                

            current_time += serve_time[route[i]]

            if current_time > vehicle_max_travel_time[vehicle_index]:
                return False

            if i > 0:
                travel_time = time_matrix[route[i - 1]][route[i]]
                current_time += travel_time

                if current_time > vehicle_max_travel_time[vehicle_index]:
                    return False

        if pickup_weight >= vehicle_capacities[vehicle_index] or current_volume > max_volume:   # or available_weight==0:
            # print("pickup weight")
            return False

        #print(pickup_weight +dropoff_weight)
        print(l)
        return True

        

    def nearest_neighbor_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, paired_orders):
        routes = [[] for _ in range(num_vehicles)]
        unvisited_customers = set(range(1, len(demands)))
        visited_customers = set()
        
        depot_index = 0  # Assuming 0 is the depot index
        
        for v in range(num_vehicles):
            current_location = depot_index
            current_route = []
            
            while unvisited_customers:
                best_distance = float('inf')
                best_pickup = None
                best_dropoff = None

                # Iterate through unvisited customers to find the nearest pickup
                for pickup, delivery in paired_orders:
                    pickup_index = int(pickup.split("_")[1])
                    delivery_index = int(delivery.split("_")[1])

                    if pickup_index in visited_customers or delivery_index in visited_customers:
                        continue

                    # Calculate the distance from the current location to the pickup
                    distance_to_pickup = distance_matrix[current_location][pickup_index]

                    # If this pickup is the closest so far and the route is valid, select it
                    if distance_to_pickup < best_distance:
                        new_route_with_pickup = current_route + [pickup_index]

                        if is_valid_route(new_route_with_pickup, demands, vehicle_capacities, order_volume, v, vehicle_volume, time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types, paired_orders):
                            best_distance = distance_to_pickup
                            best_pickup = pickup_index
                            best_dropoff = delivery_index

                # If we found a valid pickup, add it to the route along with its dropoff
                if best_pickup is not None:
                    # Add pickup to the route
                    current_route.append(best_pickup)
                    visited_customers.add(best_pickup)
                    unvisited_customers.remove(best_pickup)

                    # Now, find the corresponding dropoff location
                    current_route.append(best_dropoff)
                    visited_customers.add(best_dropoff)
                    unvisited_customers.remove(best_dropoff)

                    current_location = best_dropoff  # Move to the dropoff location

                else:
                    break  # If no valid pickup was found, exit the loop

            # Add the route for the current vehicle
            routes[v] = current_route

        return routes

    final_routes = nearest_neighbor_with_paired_orders(distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities, order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight, paired_orders)
    
    csv_data = [["Vehicle", "Route", "Route Total Orders", "Dropoff Orders", "Pickup Orders", "Route Total Weight (kg)", "Route Total Volume (cm³)", "Route Cost", "Vehicle Fixed Cost", "Route Distance Cost", "Route Time Cost", "Route Distance (km)", "Vehicle Contract Distance (km)", "Detour Distance (km)", "Vehicle Cost per Unit Distance", "Vehicle Variable Cost per Unit Distance", "Route Time (s)", "Vehicle Contract Time (s)", "Overtime (s)", "Vehicle Cost per Unit Time", "Vehicle Variable Cost per Unit Time", "Vehicle Max Travel Distance (km)", "Vehicle Max Available Time (s)", "Vehicle Max Weight (kg)", "Vehicle Max Volume (cm³)", "Vehicle Distance Utilization (%)", "Vehicle Time Utilization (%)", "Vehicle Weight Utilization (%)", "Vehicle Volume Utilization (%)"]]
    
    total_cost = 0
    total_distance = 0
    total_time = 0
    

    for i, route in enumerate(final_routes):
        if len(route) > 0:
            route_cost, route_distance, route_time = calculate_route_cost(route, i, distance_matrix, time_matrix)
            # #route_cost = calculate_route_cost(route, i, distance_matrix, time_matrix)
            
            # total_cost += route_cost
            
            # # Add start distance from depot to first customer
            # route_distance = distance_matrix[depot][route[0]] if len(route) > 0 else 0  
            
            # # Calculate the distance for the rest of the route
            # route_distance += sum(distance_matrix[route[j]][route[j + 1]] for j in range(len(route) - 1))
            
            # # Add end distance from last customer to depot
            # route_distance += distance_matrix[route[-1]][depot] if len(route) > 0 else 0
            
            # route_time += time_matrix[depot][route[0]] if len(route) > 0 else 0
            # route_time += sum(time_matrix[route[j]][route[j + 1]] for j in range(len(route) - 1))
            # route_time += time_matrix[route[-1]][depot] if len(route) > 0 else 0
            
            # # csv_data.append([i + 1, " -> ".join(map(str, route)), route_cost, route_distance, route_time])
            # # total_distance += route_distance
            # # total_time += route_time
            

            
            route_total_orders = sum(1 for j in route if j != 0)  # Exclude depot

            pickup_orders = sum(1 for j in route if order_types[j - 1] == 'pickup') if len(route) > 0 else 0
            dropoff_orders = sum(1 for j in route if order_types[j - 1] == 'dropoff') if len(route) > 0 else 0
            
            route_total_weight = sum(demands[j] for j in route if j != 0)
            route_total_volume = sum(order_volume[j] for j in route if j != 0)
            
            vehicle_distance_utilization = (route_distance / vehicle_max_travel_distance[i]) * 100
            vehicle_time_utilization = (route_time / vehicle_max_travel_time[i]) * 100
            vehicle_weight_utilization = (route_total_weight / vehicle_capacities[i]) * 100
            vehicle_volume_utilization = (route_total_volume / vehicle_volume[i]) * 100
            
            detour_distance = max(0, route_distance - contract_distance[i])
            overtime = max(0, route_time - contract_time[i])
            
            csv_data.append([
                vehicle_contract_code[i],
                " -> ".join(map(str, route)),
                route_total_orders,
                dropoff_orders,
                pickup_orders,
                route_total_weight,
                route_total_volume,
                route_cost,
                fixed_cost[i],
                cost_per_km[i]*route_distance,
                (route_time * cost_per_minute[i]),
                route_distance,
                contract_distance[i],
                detour_distance,
                cost_per_km[i],
                cost_per_km[i],
                route_time,
                contract_time[i],
                overtime,
                cost_per_minute[i],
                variable_cost_per_minute[i],
                vehicle_max_travel_distance[i],
                vehicle_max_travel_time[i],
                vehicle_max_weight[i],
                vehicle_volume[i],
                vehicle_distance_utilization,
                vehicle_time_utilization,
                vehicle_weight_utilization,
                vehicle_volume_utilization
            ])

            total_cost += route_cost
            total_distance += route_distance
            total_time += route_time

    # Save the CSV results in a specific folder
    output_dir = os.path.join(result_folder, os.path.basename(file_path).replace('.json', ''))
    os.makedirs(output_dir, exist_ok=True)

    csv_file_path = os.path.join(output_dir, 'results.csv')
    with open(csv_file_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(csv_data)

    end_time = time.time()


    print(f"Processed file: {file_path}")
    print(f"Total cost: {total_cost}")
    print(f"Total distance: {total_distance}")
    print(f"Total time: {total_time}")
    print(f"Time taken: {end_time - start_time} seconds")
    print(f"Results saved to: {csv_file_path}")




    
    end_time = time.time()
    duration = end_time - start_time

    # Prepare result output
    result = {
        "routes": final_routes,
        "duration": duration
    }

    result_file = os.path.join(result_folder, "result.json")
    with open(result_file, 'w') as f:
        json.dump(result, f, indent=4)

    print(f"Results saved to {result_file}")

if __name__ == "__main__":
    input_file = 'data1.json'  # Replace with your input file path
    result_folder = '/home/delhivery/Desktop/vrp_project/data'  # Replace with your result folder path

    if not os.path.exists(result_folder):
        os.makedirs(result_folder)

    process_vrp_file(input_file, result_folder)


['order_1_drop', 'order_2_drop', 'order_3_pickup', 'order_4_pickup', 'order_5_pickup', 'order_6_drop', 'order_7_drop', 'order_8_pickup']
pickup_weight750
[4]
Processed file: data1.json
Total cost: 4094.493846153846
Total distance: 9.16
Total time: 17034.48
Time taken: 0.0005044937133789062 seconds
Results saved to: /home/delhivery/Desktop/vrp_project/data/data1/results.csv
Results saved to /home/delhivery/Desktop/vrp_project/data/result.json


In [20]:
# Define the necessary inputs
demands = [0, 500, 299, 50, 750, 50, 100, 750, 600]
order_types = ["dropoff", "dropoff", "pickup", "pickup", "pickup", "dropoff", "dropoff", "pickup","pickup"]
vehicle_capacity = 1000  # Set vehicle capacity
vehicle_route = []
available_capacity = vehicle_capacity

# Function to check if a pickup or dropoff is possible based on vehicle capacity
def can_perform_operation(order_index, available_capacity, demands, order_types):
    if order_types[order_index] == "dropoff":
        # Dropoffs always happen if the vehicle is carrying that much load
        return True
    elif order_types[order_index] == "pickup":
        # Check if there is enough capacity for the pickup
        return available_capacity >= demands[order_index]
    return False

# Function to calculate the optimal route
def calculate_optimal_route(demands, order_types, vehicle_capacity):
    vehicle_route = []
    available_capacity = vehicle_capacity
    pickup_list = []

    # Dropoffs first, followed by pickups
    for i in range(1, len(demands)):
        if order_types[i] == "dropoff" and can_perform_operation(i, available_capacity, demands, order_types):
            vehicle_route.append(i)  # Drop off at the location
            available_capacity += demands[i]  # Increase capacity after dropoff
        elif order_types[i] == "pickup":
            pickup_list.append(i)  # Collect all pickups for processing later
    
    # Now handle the pickups
    for i in pickup_list:
        if can_perform_operation(i, available_capacity, demands, order_types):
            vehicle_route.append(i)  # Pickup at the location
            available_capacity -= demands[i]  # Decrease capacity after pickup

    return vehicle_route

# Get the optimal route
optimal_route = calculate_optimal_route(demands, order_types, vehicle_capacity)

# Print the results
print(f"Optimal vehicle route: {optimal_route}")


Optimal vehicle route: [1, 5, 6, 2, 3, 4]


In [25]:
# Define the necessary inputs
demands = [0, 500, 300, 50, 750, 50, 100, 750, 600]
order_types = ["dropoff", "dropoff", "pickup", "pickup", "pickup", "dropoff", "dropoff", "pickup"]
vehicle_capacity = 800  # Set vehicle capacity
vehicle_route = []
available_capacity = vehicle_capacity

# Function to check if a pickup or dropoff is possible based on vehicle capacity
def can_perform_operation(order_index, available_capacity, demands, order_types):
    if order_types[order_index - 1] == "dropoff":  # Adjust index for order_types
        # Dropoffs always happen if the vehicle is carrying that much load
        return True
    elif order_types[order_index - 1] == "pickup":  # Adjust index for order_types
        # Check if there is enough capacity for the pickup
        return available_capacity >= demands[order_index]
    return False

# Function to calculate the optimal route
def calculate_optimal_route(demands, order_types, vehicle_capacity):
    vehicle_route = []
    available_capacity = vehicle_capacity
    pickup_list = []
    dropoff_list = []

    # First prioritize dropoffs
    for i in range(1, len(demands)):
        if order_types[i - 1] == "dropoff":  # Adjust index for order_types
            dropoff_list.append(i)

    # Now handle pickups
    for i in range(1, len(demands)):
        if order_types[i - 1] == "pickup":  # Adjust index for order_types
            pickup_list.append(i)

    # Start with dropoffs (fully loaded initially, so we drop items first)
    for dropoff in dropoff_list:
        if can_perform_operation(dropoff, available_capacity, demands, order_types):
            vehicle_route.append(dropoff)  # Drop off at the location
            available_capacity += demands[dropoff]  # Free up capacity after dropoff

    # After dropoffs, handle pickups
    for pickup in pickup_list:
        if can_perform_operation(pickup, available_capacity, demands, order_types):
            vehicle_route.append(pickup)  # Pick up at the location
            available_capacity -= demands[pickup]  # Occupy capacity after pickup

    return vehicle_route

# Get the optimal route
optimal_route = calculate_optimal_route(demands, order_types, vehicle_capacity)

# Print the results
print(f"Optimal vehicle route: {optimal_route}")


Optimal vehicle route: [1, 2, 6, 7, 3, 4, 5, 8]


In [26]:
# I have to fix this issue like i have to make an algorithm which, an algorithm which  can  create a of a route the vehicle like route planning i have vehicle with 
# Capacity,I have dropoff order and pickup order in my route so,i have to optimize route in such a way that route is optimal 
#

In [27]:
# Data points provided
distance_matrix = [
    [0, 10, 15, 20, 10, 25, 30, 35, 40],  # Distances from depot to all locations
    [10, 0, 35, 25, 20, 15, 30, 45, 50],  # Distances from location 1
    [15, 35, 0, 30, 20, 10, 40, 50, 55],  # Distances from location 2
    [20, 25, 30, 0, 15, 35, 20, 30, 45],  # Distances from location 3
    [10, 20, 20, 15, 0, 25, 35, 40, 50],  # Distances from location 4
    [25, 15, 10, 35, 25, 0, 30, 45, 60],  # Distances from location 5
    [30, 30, 40, 20, 35, 30, 0, 50, 65],  # Distances from location 6
    [35, 45, 50, 30, 40, 45, 50, 0, 20],  # Distances from location 7
    [40, 50, 55, 45, 50, 60, 65, 20, 0]   # Distances from location 8
]

demands = [0, 500, 300, 50, 750, 50, 100, 750, 600]  # Demand at each location
order_types = ["dropoff", "dropoff", "pickup", "pickup", "pickup", "dropoff", "dropoff", "pickup"]  # Type of order
vehicle_capacity = 800  # Vehicle capacity

# Initialize vehicle route and variables
vehicle_route = [0]  # Start at the depot (index 0)
available_capacity = vehicle_capacity
current_position = 0  # Starting position is the depot
visited_orders = set()

# Helper function to find nearest neighbor
def find_nearest_order(current_position, distance_matrix, demands, available_capacity, order_types, visited_orders):
    nearest_order = None
    nearest_distance = float('inf')

    # Iterate over all locations to find the nearest unvisited order
    for i in range(1, len(distance_matrix)):
        if i not in visited_orders:
            order_type = order_types[i - 1]
            if (order_type == "dropoff") or (order_type == "pickup" and demands[i] <= available_capacity):
                distance = distance_matrix[current_position][i]
                if distance < nearest_distance:
                    nearest_order = i
                    nearest_distance = distance
    return nearest_order, nearest_distance

# Route planning algorithm
while len(visited_orders) < len(demands) - 1:  # -1 to exclude depot from the count
    # Find the nearest pickup or dropoff location
    nearest_order, nearest_distance = find_nearest_order(current_position, distance_matrix, demands, available_capacity, order_types, visited_orders)

    # If no order is found, return to depot to unload or load
    if nearest_order is None:
        vehicle_route.append(0)  # Return to depot
        current_position = 0
        available_capacity = vehicle_capacity  # Reset capacity
        continue

    # Add the nearest order to the route
    vehicle_route.append(nearest_order)
    visited_orders.add(nearest_order)

    # Process the order: either pickup or dropoff
    if order_types[nearest_order - 1] == "pickup":
        # Pickup the order and reduce vehicle capacity
        available_capacity -= demands[nearest_order]
        pickup_weight+=demands[nearest_order]
        
        print(f"Picking up from location {nearest_order}, remaining capacity: {available_capacity}")
    else:
        # Dropoff the order, freeing up capacity
        available_capacity += demands[nearest_order]
        print(f"Dropping off at location {nearest_order}, remaining capacity: {available_capacity}")

    # Move to the next position
    current_position = nearest_order

# Finally, return to the depot after all orders are serviced
vehicle_route.append(0)

# Print the final route
print("Final Vehicle Route:", vehicle_route)


Dropping off at location 1, remaining capacity: 1300
Picking up from location 5, remaining capacity: 1250
Dropping off at location 2, remaining capacity: 1550
Picking up from location 4, remaining capacity: 800
Picking up from location 3, remaining capacity: 750
Dropping off at location 6, remaining capacity: 850
Dropping off at location 7, remaining capacity: 1600
Picking up from location 8, remaining capacity: 1000
Final Vehicle Route: [0, 1, 5, 2, 4, 3, 6, 7, 8, 0]


In [7]:

orders = [
    {'order_id': 1, 'type': 'Dropoff', 'weight': 500, 'paired_order': 2},
    {'order_id': 2, 'type': 'Dropoff', 'weight': 200, 'paired_order': 1},
    {'order_id': 3, 'type': 'Pickup', 'weight': 200, 'paired_order': None},
    {'order_id': 4, 'type': 'Pickup', 'weight': 750, 'paired_order': 7},
    {'order_id': 5, 'type': 'Pickup', 'weight': 50, 'paired_order': None},
    {'order_id': 6, 'type': 'Dropoff', 'weight': 100, 'paired_order': None},
    {'order_id': 7, 'type': 'Dropoff', 'weight': 750, 'paired_order': 4},
    {'order_id': 8, 'type': 'Pickup', 'weight': 600, 'paired_order': None}
]


vehicle_capacity = 800
current_load = 0
pickup_weight=0
dropoff_weight=0
available_capacity=800

planned_route = [1, 2, 3, 4, 5, 6, 7, 8]


vehicle_route = []


for order_id in planned_route:
    
    order = next((o for o in orders if o['order_id'] == order_id), None)
    
    if order:
        if order['type'] == 'Pickup':
            if pickup_weight + order['weight'] <= vehicle_capacity:
                

                pickup_weight += order['weight']
                print(pickup_weight)
                
                vehicle_route.append(order['order_id'])
                available_capacity-=order['weight']
                
                
                
            else:
                print('hi')

        elif order['type'] == 'Dropoff':
            if order['paired_order'] in vehicle_route:

            
                vehicle_route.append(order['order_id'])
                available_capacity += order['weight']
                pickup_weight-=order['weight']

            else:
                if pickup_weight+dropoff_weight<=800:
                    dropoff_weight+=order['weight']
                    vehicle_route.append(order['order_id'])
                    

    else:
        print(f"Order {order_id} not found.")



print(vehicle_route)




0
750
800
650
[1, 2, 3, 4, 5, 7, 8]


In [25]:
# Define the orders
orders = [
    {'order_id': 1, 'type': 'Dropoff', 'weight': 500, 'paired_order': 2},
    {'order_id': 2, 'type': 'Dropoff', 'weight': 300, 'paired_order': 1},
    {'order_id': 3, 'type': 'Pickup', 'weight': 500, 'paired_order': None},
    {'order_id': 4, 'type': 'Pickup', 'weight': 750, 'paired_order': 7},
    {'order_id': 5, 'type': 'Pickup', 'weight': 50, 'paired_order': None},
    {'order_id': 6, 'type': 'Dropoff', 'weight': 100, 'paired_order': None},
    {'order_id': 7, 'type': 'Dropoff', 'weight': 750, 'paired_order': 4},
    {'order_id': 8, 'type': 'Pickup', 'weight': 600, 'paired_order': None}
]

# Vehicle details
vehicle_capacity = 800
vehicles = [
    {'id': 1, 'current_load': 0, 'available_capacity': vehicle_capacity, 'route': [], 'pickup_weight': 0, 'dropoff_weight': 0},
    {'id': 2, 'current_load': 0, 'available_capacity': vehicle_capacity, 'route': [], 'pickup_weight': 0, 'dropoff_weight': 0}
]

# Planned route for testing purposes (can be dynamic)
planned_route = [1, 2, 3, 4, 5, 6, 7, 8]

# Function to assign an order to the best suitable vehicle
def assign_order_to_vehicle(order, vehicles):
    for vehicle in vehicles:
        if order['type'] == 'Pickup':
            if vehicle['pickup_weight'] <= vehicle_capacity:
                # Add pickup order to the vehicle
                vehicle['pickup_weight'] += order['weight']
                vehicle['available_capacity'] -= order['weight']
                vehicle['route'].append(order['order_id'])
                return True

        elif order['type'] == 'Dropoff':
            # Check if the paired order is already in the route
            if order['paired_order'] in vehicle['route']:
                vehicle['available_capacity'] += order['weight'] # I debug one thing is that phone is distracting me 
                vehicle['pickup_weight'] -= order['weight']
                vehicle['route'].append(order['order_id'])
                return True
            else:
                if vehicle['pickup_weight'] + vehicle['dropoff_weight'] + order['weight'] <= vehicle_capacity:
                    vehicle['dropoff_weight'] += order['weight']
                    vehicle['route'].append(order['order_id'])
                    return True

    return False

# Process each order in the planned route
for order_id in planned_route:
    order = next((o for o in orders if o['order_id'] == order_id), None)

    if order:
        assigned = assign_order_to_vehicle(order, vehicles)
        if not assigned:
            print(f"Order {order['order_id']} could not be assigned to any vehicle due to capacity constraints.")
    else:
        print(f"Order {order_id} not found.")

# Print the routes of each vehicle
for vehicle in vehicles:
    print(f"Vehicle {vehicle['id']} route: {vehicle['route']}")


Vehicle 1 route: [1, 2, 3, 4, 7, 8]
Vehicle 2 route: [5, 6]


In [20]:
# Define orders with paired relationships
orders = [
    {'order_id': 1, 'type': 'Dropoff', 'weight': 500, 'paired_order': 1},
    {'order_id': 2, 'type': 'Dropoff', 'weight': 300, 'paired_order': 1},
    {'order_id': 3, 'type': 'Pickup', 'weight': 5, 'paired_order': None},
    {'order_id': 4, 'type': 'Pickup', 'weight': 750, 'paired_order': 7},
    {'order_id': 5, 'type': 'Pickup', 'weight': 500, 'paired_order': None},
    {'order_id': 6, 'type': 'Dropoff', 'weight': 100, 'paired_order': None},
    {'order_id': 7, 'type': 'Dropoff', 'weight': 750, 'paired_order': 4},
    {'order_id': 8, 'type': 'Pickup', 'weight': 600, 'paired_order': None}
]

vehicle_capacity = 800
current_load = 0
pickup_weight = 0
dropoff_weight = 0
available_capacity = vehicle_capacity

planned_route = [1, 2, 3, 4, 5, 6, 7, 8]
# print(order['paired_order'])
vehicle_route = []
paired_orders = {}  # Dictionary to track paired pickups and drop-offs

# Create a dictionary to track pickups and their associated drop-offs
for order in orders:
    if order['type'] == 'Pickup' and order['paired_order']:
        # Identify paired drop-off for this pickup
        paired_order = next((o for o in orders if o['order_id'] == order['paired_order']), None)
        if paired_order:
            paired_orders[order['order_id']] = paired_order['order_id']

# Main route processing
for order_id in planned_route:
    order = next((o for o in orders if o['order_id'] == order_id), None)

    if order:
        if order['type'] == 'Pickup':
            paired_dropoff_id = paired_orders.get(order['order_id'])
            paired_dropoff = next((o for o in orders if o['order_id'] == paired_dropoff_id), None)

            # Check if both pickup and its paired drop-off fit within the capacity
            if paired_dropoff and (order['weight']  <= vehicle_capacity):
                # Add both pickup and drop-off to the route
                vehicle_route.append(order['order_id'])
                vehicle_route.append(paired_dropoff['order_id'])
                pickup_weight += order['weight']
                dropoff_weight += paired_dropoff['weight']
                available_capacity -= order['weight']
            elif not paired_dropoff:  # Handle pickups without paired drop-offs
                
                if pickup_weight + order['weight'] <= vehicle_capacity:
                    vehicle_route.append(order['order_id'])
                    pickup_weight += order['weight']
                    available_capacity -= order['weight']
            else:
                print(f"Skipping order {order['order_id']} and paired drop-off {paired_dropoff['order_id']} due to capacity constraints.")

        elif order['type'] == 'Dropoff':
            # Check if paired pickup is already in the route
            if order['paired_order'] in vehicle_route:
                if dropoff_weight + order['weight'] <= vehicle_capacity:
                    vehicle_route.append(order['order_id'])
                    available_capacity += order['weight']
                    # pickup_weight -= order['weight']  # Reduce the pickup weight
                    # print(pickup_weight)
                else:
                    print(f"Skipping drop-off {order['order_id']} due to capacity constraints.")
            else:
                # Drop-offs without paired pickups in the route
                if pickup_weight + dropoff_weight <= vehicle_capacity:
                    dropoff_weight += order['weight']
                    vehicle_route.append(order['order_id'])

    else:
        print(f"Order {order_id} not found.")

print("Final Vehicle Route:", vehicle_route)


-300
Skipping drop-off 7 due to capacity constraints.
Final Vehicle Route: [1, 2, 3, 4, 7]


In [102]:
l="kush"
print(len(l))

4


In [104]:
i=j=0
print(i)

0


In [106]:
l=[1,2,3,44]
print(max(l))

44


In [113]:
l=[1,0,0,0,0,1]
n=1
# for i in range(len(l)):
if l[n-1]==1 and l[n+1] ==1:
    print(True)
else:
    print(False)

False


In [125]:
l="kushagra"
#print(l[::-1])
v=["a","e","i","o","u"]

for i in range(len(l)):
    if l[i] in v:
        print(l[i])
        

argahsuk
u
a
a


In [126]:
def reverse_vowels(word):
    vowels = "aeiouAEIOU"  # Define vowels, including uppercase for generality
    word = list(word)       # Convert the word to a list to allow modifications
    i, j = 0, len(word) - 1 # Set pointers at the start and end of the list
    
    # Use two pointers to swap vowels in the word
    while i < j:
        # Move the start pointer until it finds a vowel
        while i < j and word[i] not in vowels:
            i += 1
        # Move the end pointer until it finds a vowel
        while i < j and word[j] not in vowels:
            j -= 1
        
        # Swap the vowels found by the two pointers
        word[i], word[j] = word[j], word[i]
        i += 1
        j -= 1
    
    # Join the list back into a string
    return ''.join(word)

# Example usage
word = "kushagra"
reversed_vowels = reverse_vowels(word)
print(reversed_vowels)  # Output will be: "kishugra"


kashagru


In [ ]:

for i in range(len(l)):


In [142]:
l=[1,2,3,4,5]
# i=1
# print(l[i+1]*l[i+2]*l[i+3])
j=0
m=[]
while j<len(l)-2:
    
    m.append(l[j]*l[j+1]*[j+2])
    i+=1
    j+=1
print(m)


[[2, 2], [3, 3, 3, 3, 3, 3], [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]


In [144]:
l=[1,2,3,4]
r=[]
for i in range(len(l)):
    for j in range(i+1,len(l)):
        r.append(l[i]*l[j])
print(r)



[2, 3, 4, 6, 8, 12]


In [145]:
from functools import reduce

l = [1, 2, 3, 4]
result = []

for i in range(len(l)):
    # Create a sublist that excludes the current element
    sublist = l[:i] + l[i+1:]
    # Multiply all elements of the sublist
    product = reduce(lambda x, y: x * y, sublist)
    result.append(product)

print(result)


[24, 12, 8, 6]


In [147]:
l = [1, 2, 3, 4]
result = []

for i in range(len(l)):
    product = 1  # Initialize product as 1
    for j in range(len(l)):
        if i != j:  # Skip the current element
            product *= l[j]
    result.append(product)

print(result)


[1, 2, 3, 4]


In [149]:
l=[1,2,3,4]
r=[]

for i in range(len(l)):
    s=0
    for j in range(len(l)):
        if i!=j:
            s+=l[j]
    r.append(s)
print(r)

[9, 8, 7, 6]


In [156]:
l=[1,2,3,4,5]
r=[]
m=max(l)
for i in range(len(l)):
    s=0
    for i in range(len(l)):
        if i!=j and m>l[j]:
            
    r.append(m)
            
            
print(r)


IndentationError: expected an indented block (3814862229.py, line 9)

In [157]:
l = [5, 1, 3, 9]
result = []

for i in range(len(l)):
    max_value = float('-inf')  # Initialize with the smallest possible value
    for j in range(len(l)):
        if i != j:  # Skip the current element
            if l[j] > max_value:
                max_value = l[j]  # Update max_value if a larger element is found
    result.append(max_value)

print(result)



[9, 9, 9, 5]


In [172]:
c = ["a","b","b","b"]
u=["a","b","c","c"]
s=0
for i in range(len(c)):
    for j in range(i+1,len(c)):
        if c[j]==c[i]:
            s+=1
print(s)


3


In [170]:
print(set(u))

{'b', 'a', 'c'}


In [178]:
# Given list
c = ["a", "b", "b", "b","b"]

# Initialize an empty dictionary to store counts
element_count = {}

# Iterate through the list to count each element
for i in c:
    if i in element_count:
        element_count[i] += 1
    else:
        element_count[i] = 1

# Display the frequency of each element
print([element_count])  # Output: {'a': 1, 'b': 3}


[{'a': 1, 'b': 4}]


In [187]:
l="kushagraaaaaaaa"
d={}
for i in l:
    if i in d:
        d[i]+=1
    else:
        d[i]=1
print(d)


{'k': 1, 'u': 1, 's': 1, 'h': 1, 'a': 9, 'g': 1, 'r': 1}


In [200]:
# Using items() to get key-value pairs
l=[]
for i, j in d.items():
    # print(f'Key: {i}, Value: {value}')
    # if j >1:
    #     print(i,j)
    l.append(j)
print(max(l))


9


In [203]:
l=[1,2,3,4,5]
m=max(l)
n=0
for i in range(len(l)):
    if 3<l[i]<5:
        print(l[i])


<built-in method sort of list object at 0x7f287484d440>
4


In [204]:
# Example list
numbers = [4, 1, 3, 9, 2, 8]

# Sort the list and get the second largest
unique_numbers = list(set(numbers))  # Remove duplicates
unique_numbers.sort()  # Sort the list
second_largest = unique_numbers[-2]  # Access the second largest

print(second_largest)  # Output: 8


8


In [205]:
# Example list
numbers = [4, 1, 3, 9, 2, 8]

# Initialize the first and second largest
first_largest = second_largest = 0

for num in numbers:
    if num > first_largest:
        second_largest = first_largest
        first_largest = num
    elif first_largest > num > second_largest:
        second_largest = num

print(second_largest)  # Output: 8


8


In [2]:
from datetime import datetime, timedelta

def local_cheapest_insertion_with_paired_orders(
    distance_matrix, time_matrix, num_vehicles, demands, vehicle_capacities,
    order_volume, vehicle_max_travel_time, serve_time, vehicle_max_weight,
    vehicle_volume, order_types, paired_orders, paired_pickup_orders,
    paired_dropoff_orders, vehicle_time_windows, order_time_windows,
    vehicle_max_travel_distance, vehicle_max_order):
    
    routes = [[] for _ in range(num_vehicles)]
    skipped_orders = []
    unvisited_customers = set(range(1, len(demands)))  # Assuming customer IDs start from 1

    # Dictionaries for paired orders
    pairing_pickup_to_dropoff = {pair[0]: pair[1] for pair in paired_orders}  # Pickup -> Dropoff
    paired_pickup_dict = {pair[0]: pair[1] for pair in paired_pickup_orders}  # Pickup -> Pickup
    paired_dropoff_dict = {pair[0]: pair[1] for pair in paired_dropoff_orders}  # Dropoff -> Dropoff

    # Iterate over vehicles one by one
    for v in range(num_vehicles):
        if not unvisited_customers:
            print(f"All customers have been assigned. No unvisited customers left for vehicle {v}.")
            break  # All customers have been assigned

        # Get vehicle's capacity and time window
        vehicle_capacity = vehicle_capacities[v]
        vehicle_start_time, vehicle_end_time = vehicle_time_windows[v]
        current_time = vehicle_start_time
        current_route = []
        pickup_weight = 0
        dropoff_weight = 0
        available_weight = vehicle_capacity

        print(f"Assigning customers to Vehicle {v}")

        # Iterate through a list to allow modification during iteration
        for order in list(unvisited_customers):
            if order in skipped_orders:
                continue  # Skip already skipped orders

            ot = order_types[order - 1]  # Adjust for zero-based indexing
            weight = demands[order]

            if ot == "pickup":
                # Handle paired pickup orders
                if order in paired_pickup_orders:
                    paired_pickup = paired_pickup_orders[order]
                    paired_weight = demands[paired_pickup]

                    if paired_pickup not in unvisited_customers:
                        # Paired pickup already assigned
                        skipped_orders.extend([order, paired_pickup])
                        print(f"Skipped Pickup Order {order} and Paired Pickup {paired_pickup} as paired pickup is already assigned.")
                        continue

                    total_pickup_weight = weight + paired_weight
                    if pickup_weight + total_pickup_weight <= vehicle_capacity and available_weight >= total_pickup_weight:
                        # Assign both pickup and its paired pickup
                        current_route.extend([order, paired_pickup])
                        pickup_weight += total_pickup_weight
                        available_weight -= total_pickup_weight
                        unvisited_customers.remove(order)
                        unvisited_customers.remove(paired_pickup)
                        print(f"Assigned Paired Pickup Orders {order} and {paired_pickup} to current route.")
                    else:
                        # Cannot assign due to capacity constraints
                        skipped_orders.extend([order, paired_pickup])
                        print(f"Skipped Paired Pickup Orders {order} and {paired_pickup} due to capacity constraints.")
                        continue

                # Handle pickup paired with dropoff
                elif order in pairing_pickup_to_dropoff:
                    paired_dropoff = pairing_pickup_to_dropoff[order]
                    paired_dropoff_weight = demands[paired_dropoff]

                    if paired_dropoff not in unvisited_customers:
                        # Paired dropoff already assigned
                        skipped_orders.extend([order, paired_dropoff])
                        print(f"Skipped Pickup Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
                        continue

                    if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
                        # Assign pickup and its paired dropoff
                        current_route.extend([order, paired_dropoff])
                        pickup_weight += weight
                        dropoff_weight += paired_dropoff_weight
                        available_weight -= (weight + paired_dropoff_weight)
                        unvisited_customers.remove(order)
                        unvisited_customers.remove(paired_dropoff)
                        print(f"Assigned Pickup Order {order} and its Paired Dropoff Order {paired_dropoff} to current route.")
                    else:
                        # Cannot assign due to capacity constraints
                        skipped_orders.extend([order, paired_dropoff])
                        print(f"Skipped Pickup Order {order} and Dropoff Order {paired_dropoff} due to capacity constraints.")
                        continue

                else:
                    # Unpaired pickup
                    if pickup_weight + weight <= vehicle_capacity and available_weight >= weight:
                        # Assign unpaired pickup
                        current_route.append(order)
                        pickup_weight += weight
                        available_weight -= weight
                        unvisited_customers.remove(order)
                        print(f"Assigned Unpaired Pickup Order {order} to current route.")
                    else:
                        # Cannot assign due to capacity constraints
                        skipped_orders.append(order)
                        print(f"Skipped Unpaired Pickup Order {order} due to capacity constraints.")
                        continue

            elif ot == "dropoff":
                # Handle paired dropoff orders
                paired_dropoff = None
                for key, value in paired_dropoff_dict.items():
                    if value == order:
                        paired_dropoff = key
                        break
                    elif key == order:
                        paired_dropoff = value
                        break

                if paired_dropoff:
                    paired_dropoff_weight = demands[paired_dropoff]

                    if paired_dropoff not in unvisited_customers:
                        # Paired dropoff already assigned
                        skipped_orders.extend([order, paired_dropoff])
                        print(f"Skipped Dropoff Order {order} and Paired Dropoff {paired_dropoff} as paired dropoff is already assigned.")
                        continue

                    if dropoff_weight + weight + paired_dropoff_weight <= vehicle_capacity and available_weight >= (weight + paired_dropoff_weight):
                        # Assign both dropoff and its paired dropoff
                        current_route.extend([order, paired_dropoff])
                        dropoff_weight += (weight + paired_dropoff_weight)
                        available_weight -= (weight + paired_dropoff_weight)
                        unvisited_customers.remove(order)
                        unvisited_customers.remove(paired_dropoff)
                        print(f"Assigned Paired Dropoff Orders {order} and {paired_dropoff} to current route.")
                    else:
                        # Cannot assign due to capacity constraints
                        skipped_orders.extend([order, paired_dropoff])
                        print(f"Skipped Paired Dropoff Orders {order} and {paired_dropoff} due to capacity constraints.")
                        continue
                else:
                    # Unpaired dropoff
                    if dropoff_weight + weight <= vehicle_capacity and available_weight >= weight:
                        # Assign unpaired dropoff
                        current_route.append(order)
                        dropoff_weight += weight
                        available_weight -= weight
                        unvisited_customers.remove(order)
                        print(f"Assigned Unpaired Dropoff Order {order} to current route.")
                    else:
                        # Cannot assign due to capacity constraints
                        skipped_orders.append(order)
                        print(f"Skipped Unpaired Dropoff Order {order} due to capacity constraints.")
                        continue

        if current_route:
            routes[v] = current_route
            print(f"Finalized Route for Vehicle {v}: {current_route}")

    return routes, skipped_orders

def is_valid_route(route, demands, vehicle_capacities, order_volume, vehicle_index, vehicle_volume,
                time_matrix, vehicle_max_travel_time, serve_time, vehicle_max_weight, order_types,
                pairing_pickup_to_dropoff, paired_pickup_orders, paired_dropoff_orders,
                vehicle_time_windows, order_time_windows, current_travel_time,
                max_travel_time, current_travel_distance, max_travel_distance,
                current_order_count, max_order_count):
    # Initialize variables
    vehicle_start_time, vehicle_end_time = vehicle_time_windows[vehicle_index]
    current_time = vehicle_start_time
    available_weight = vehicle_capacities[vehicle_index]
    current_volume = 0
    pickup_weight = 0
    dropoff_weight = 0
    assigned_dropoffs = set()
    assigned_pickups = set()
    total_travel_time_increment = timedelta(seconds=0)
    total_travel_distance_increment = 0
    order_count_increment = 0

    for i, order in enumerate(route):
        if order in assigned_dropoffs or order in assigned_pickups:
            continue

        ot = order_types[order - 1]  # Adjust for zero-based indexing
        weight = demands[order]

        if ot == "pickup":
            if order in paired_pickup_orders:
                paired_pickup = paired_pickup_orders[order]
                paired_weight = demands[paired_pickup]

                if paired_pickup not in assigned_pickups:
                    total_pickup_weight = weight + paired_weight
                    if pickup_weight + total_pickup_weight <= vehicle_capacities[vehicle_index] and available_weight >= total_pickup_weight:
                        assigned_pickups.update([order, paired_pickup])
                        pickup_weight += total_pickup_weight
                        available_weight -= total_pickup_weight
                    else:
                        return False, None, None  # Capacity exceeded with paired pickup
            elif order in pairing_pickup_to_dropoff:
                paired_dropoff = pairing_pickup_to_dropoff[order]
                paired_dropoff_weight = demands[paired_dropoff]

                if paired_dropoff not in assigned_dropoffs:
                    if pickup_weight + weight + paired_dropoff_weight <= vehicle_capacities[vehicle_index] and available_weight >= (weight + paired_dropoff_weight):
                        assigned_pickups.add(order)
                        assigned_dropoffs.add(paired_dropoff)
                        pickup_weight += weight
                        dropoff_weight += paired_dropoff_weight
                        available_weight -= (weight + paired_dropoff_weight)
                    else:
                        return False, None, None  # Capacity exceeded with pickup and its paired dropoff
            else:
                if pickup_weight + weight <= vehicle_capacities[vehicle_index] and available_weight >= weight:
                    assigned_pickups.add(order)
                    pickup_weight += weight
                    available_weight -= weight
                else:
                    return False, None, None  # Capacity exceeded with unpaired pickup

        elif ot == "dropoff":
            # Find paired dropoff
            paired_dropoff = None
            for key, value in paired_dropoff_orders.items():
                if value == order:
                    paired_dropoff = key
                    break
                elif key == order:
                    paired_dropoff = value
                    break

            if paired_dropoff:
                paired_dropoff_weight = demands[paired_dropoff]

                if paired_dropoff not in assigned_dropoffs:
                    if dropoff_weight + weight + paired_dropoff_weight <= vehicle_capacities[vehicle_index] and available_weight >= (weight + paired_dropoff_weight):
                        assigned_dropoffs.update([order, paired_dropoff])
                        dropoff_weight += (weight + paired_dropoff_weight)
                        available_weight -= (weight + paired_dropoff_weight)
                    else:
                        return False, None, None  # Capacity exceeded with paired dropoff
            else:
                if dropoff_weight + weight <= vehicle_capacities[vehicle_index] and available_weight >= weight:
                    assigned_dropoffs.add(order)
                    dropoff_weight += weight
                    available_weight -= weight
                else:
                    return False, None, None  # Capacity exceeded with unpaired dropoff

        # Update order count
        order_count_increment += 1
        if order_count_increment > max_order_count:
            return False, None, None  # Exceeds max order count

        # Update travel time and distance increment
        if i == 0:
            # From depot to first customer
            travel_time = timedelta(seconds=time_matrix[0][order])
            travel_distance = distance_matrix[0][order]
        else:
            prev_order = route[i - 1]
            travel_time = timedelta(seconds=time_matrix[prev_order][order])
            travel_distance = distance_matrix[prev_order][order]

        current_time += travel_time
        total_travel_time_increment += travel_time
        total_travel_distance_increment += travel_distance

        # Check time window constraints
        order_start_time, order_end_time = order_time_windows[order - 1]
        if current_time > order_end_time:
            return False, None, None  # Cannot arrive before time window closes
        if current_time < order_start_time:
            current_time = order_start_time  # Wait until time window opens

        # Add service time
        service_time = timedelta(seconds=serve_time[order])
        current_time += service_time
        total_travel_time_increment += service_time

        # Check vehicle's operating time window
        if current_time > vehicle_end_time:
            return False, None, None  # Exceeds vehicle's operating time window

    # After processing all orders, check final constraints on travel time and distance
    if (current_travel_time + total_travel_time_increment > timedelta(seconds=max_travel_time) or
        current_travel_distance + total_travel_distance_increment > max_travel_distance):
        return False, None, None

    return True, total_travel_time_increment, total_travel_distance_increment

def calculate_insertion_cost(distance_matrix, current_route, customer, position):
    if position == 0:
        prev_node = 0  # Depot
    else:
        prev_node = current_route[position - 1]
    if position == len(current_route):
        next_node = 0  # Depot or end of route
    else:
        next_node = current_route[position]

    additional_cost = (
        distance_matrix[prev_node][customer] +
        distance_matrix[customer][next_node] -
        distance_matrix[prev_node][next_node]
    )
    return additional_cost

# Example usage
if __name__ == "__main__":
    # Example data (replace with actual data)
    distance_matrix = [
        [0, 10, 20, 30],  # Depot (0) to customers 1, 2, 3
        [10, 0, 15, 25],
        [20, 15, 0, 35],
        [30, 25, 35, 0]
    ]
    
    time_matrix = [
        [0, 600, 1200, 1800],  # Depot to customers (in seconds)
        [600, 0, 900, 1500],
        [1200, 900, 0, 2100],
        [1800, 1500, 2100, 0]
    ]
    
    num_vehicles = 2
    demands = [0, 5, 10, 15]  # Depot has 0 demand, customers 1, 2, 3
    vehicle_capacities = [20, 20]  # Two vehicles with capacity 20 each
    order_volume = [0, 100, 200, 300]  # Volume in cm³
    vehicle_max_travel_time = [3600, 3600]  # 1 hour in seconds
    serve_time = [0, 300, 300, 300]  # Service time in seconds
    vehicle_max_weight = [20, 20]  # Max weight in kg
    vehicle_volume = [1000, 1000]  # Max volume in cm³
    order_types = ["depot", "pickup", "pickup", "dropoff"]  # Including depot
    paired_orders = [(1, 3)]  # Pickup 1 is paired with Dropoff 3
    paired_pickup_orders = []  # No paired pickups in this example
    paired_dropoff_orders = []  # No paired dropoffs in this example
    vehicle_time_windows = [
        (datetime(2024, 10, 4, 8, 0, 0), datetime(2024, 10, 4, 12, 0, 0)),
        (datetime(2024, 10, 4, 9, 0, 0), datetime(2024, 10, 4, 13, 0, 0))
    ]
    order_time_windows = [
        (datetime(2024, 10, 4, 8, 30, 0), datetime(2024, 10, 4, 10, 30, 0)),  # Customer 1
        (datetime(2024, 10, 4, 9, 0, 0), datetime(2024, 10, 4, 11, 0, 0)),   # Customer 2
        (datetime(2024, 10, 4, 10, 0, 0), datetime(2024, 10, 4, 12, 0, 0))   # Customer 3
    ]
    vehicle_max_travel_distance = [100, 100]  # in km
    vehicle_max_order = [5, 5]  # Maximum number of orders per vehicle

    # Call the function
    final_routes, skipped_orders = local_cheapest_insertion_with_paired_orders(
        distance_matrix=distance_matrix,
        time_matrix=time_matrix,
        num_vehicles=num_vehicles,
        demands=demands,
        vehicle_capacities=vehicle_capacities,
        order_volume=order_volume,
        vehicle_max_travel_time=vehicle_max_travel_time,
        serve_time=serve_time,
        vehicle_max_weight=vehicle_max_weight,
        vehicle_volume=vehicle_volume,
        order_types=order_types,
        paired_orders=paired_orders,
        paired_pickup_orders=paired_pickup_orders,
        paired_dropoff_orders=paired_dropoff_orders,
        vehicle_time_windows=vehicle_time_windows,
        order_time_windows=order_time_windows,
        vehicle_max_travel_distance=vehicle_max_travel_distance,
        vehicle_max_order=vehicle_max_order
    )
    print("Final Routes:", final_routes)
    print("Skipped Orders:", skipped_orders)


Assigning customers to Vehicle 0
Assigned Unpaired Pickup Order 2 to current route.
Skipped Unpaired Pickup Order 3 due to capacity constraints.
Finalized Route for Vehicle 0: [2]
Assigning customers to Vehicle 1
Final Routes: [[2], []]
Skipped Orders: [3]
